In [1]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [2]:


!nvidia-smi -L
!nvidia-smi
!lscpu |grep 'Model name'
# !lscpu 
!free -h --si | awk  '/Mem:/{print $2}'

/bin/bash: nvidia-smi: command not found
/bin/bash: nvidia-smi: command not found
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
12G


In [3]:
!pip install transformers datasets
!pip install transformers
!pip install 'transformers[torch]'
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00
Looking in indexe

In [4]:
from transformers import AutoTokenizer, BertForMaskedLM, BertForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
mmodel = BertForMaskedLM.from_pretrained("bert-base-uncased")
scmodel = BertForSequenceClassification.from_pretrained("bert-base-uncased")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")

with torch.no_grad():
    logits = mmodel(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
predicted_words=tokenizer.decode(predicted_token_id)

print(f"Predicted words: {predicted_words}")

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = mmodel(**inputs, labels=labels)
round(outputs.loss.item(), 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.Lay

Predicted words: paris


0.88

In [ ]:
outputs

MaskedLMOutput(loss=tensor(0.8752, grad_fn=<NllLossBackward0>), logits=tensor([[[ -6.4346,  -6.4063,  -6.4097,  ...,  -5.7691,  -5.6326,  -3.7883],
         [-14.0119, -14.7240, -14.2120,  ..., -11.6976, -10.7304, -12.7617],
         [ -9.6561, -10.3125,  -9.7459,  ...,  -8.7782,  -6.6036, -12.6596],
         ...,
         [ -3.7861,  -3.8572,  -3.5644,  ...,  -2.5593,  -3.1093,  -4.3820],
         [-11.6598, -11.4274, -11.9266,  ...,  -9.8772, -10.2103,  -4.7594],
         [-11.7267, -11.7509, -11.8040,  ..., -10.5943, -10.9407,  -7.5151]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
mmodel, scmodel

(BertForMaskedLM(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-11): 12 x BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): LayerNorm((768,), ep

In [ ]:
predicted_token_id

tensor([3000])

In [ ]:
tokenizer.decode(predicted_token_id)

'paris'

In [ ]:
inputs

{'input_ids': tensor([[ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
logits.shape

torch.Size([1, 9, 30522])

In [ ]:
mask_token_index

tensor([6])

In [ ]:
inputs["input_ids"][0]

tensor([ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102])

In [ ]:
words = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [ ]:
words

['[CLS]', 'the', 'capital', 'of', 'france', 'is', '[MASK]', '.', '[SEP]']

In [ ]:
labels

tensor([[-100, -100, -100, -100, -100, -100, 3000, -100, -100]])

In [ ]:
from transformers import pipeline

fill_masker = pipeline(model="bert-base-uncased")
fill_masker("This is a simple [MASK].")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.04227164387702942,
  'token': 3291,
  'token_str': 'problem',
  'sequence': 'this is a simple problem.'},
 {'score': 0.031049933284521103,
  'token': 3160,
  'token_str': 'question',
  'sequence': 'this is a simple question.'},
 {'score': 0.029722534120082855,
  'token': 8522,
  'token_str': 'equation',
  'sequence': 'this is a simple equation.'},
 {'score': 0.026522196829319,
  'token': 2028,
  'token_str': 'one',
  'sequence': 'this is a simple one.'},
 {'score': 0.024157337844371796,
  'token': 3627,
  'token_str': 'rule',
  'sequence': 'this is a simple rule.'}]

In [ ]:
classifier = pipeline("text-classification", model=scmodel, tokenizer=tokenizer)  #"fill-mask"
sentences = [
    "The judge told the jurors to think fast.\n And I had to go straight to the point.",
    "The judge told that the jurors to think fast."
]
classifier(sentences)

[{'label': 'LABEL_1', 'score': 0.504574179649353},
 {'label': 'LABEL_1', 'score': 0.524419367313385}]

In [ ]:
fillmasker = pipeline("fill-mask", model=mmodel, tokenizer=tokenizer)  #
sentences = [
    "I [MASK] [MASK] milk.",
    "I [MASK] like drinking [MASK]."
]
fillmasker(sentences)

NameError: ignored

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Replace these lists with your actual dataset
sentences = ["This is an example sentence from my domain.", "Another example sentence from my domain."]

# Load the tokenizer and the pre-trained model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# Tokenize the dataset
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

# Initialize the dataset
dataset = CustomDataset(inputs)

# Set up the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10,
    seed=42,
)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-25-4f4d21cb1f7d>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=3, training_loss=4.320665677388509, metrics={'train_runtime': 10.332, 'train_samples_per_second': 0.581, 'train_steps_per_second': 0.29, 'total_flos': 33928745400.0, 'train_loss': 4.320665677388509, 'epoch': 3.0})

In [ ]:
torch.save(mmodel, "maskedmodel.pth")

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.eval()  # Set the model to evaluation mode

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [5]:
def predict_masked_tokens(text, tokenizer, model, top_k=3):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    mask_positions = torch.where(input_ids == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        output = model(input_ids)
    logits = output.logits

    predicted_tokens = []
    for mask_position in mask_positions:
        probs = logits[0, mask_position].softmax(dim=-1)
        top_k_values, top_k_indices = torch.topk(probs, top_k)
        top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices)
        predicted_tokens.append(top_k_tokens)

    return predicted_tokens


In [6]:
text = "1801—I have just returned from a [MASK] to my landlord—the solitary neighbour that I shall be troubled [MASK]."
predicted_tokens = predict_masked_tokens(text, tokenizer, mmodel, top_k=5)
predicted_tokens

[['visit', 'letter', 'call', 'message', 'talk'],
 ['by', 'about', 'with', 'of', 'from']]

In [7]:
import re

replaced_text = re.sub(r"\[MASK\]", "{}", text)
replaced_text = replaced_text.format(*predicted_tokens)


In [8]:
replaced_text

"1801—I have just returned from a ['visit', 'letter', 'call', 'message', 'talk'] to my landlord—the solitary neighbour that I shall be troubled ['by', 'about', 'with', 'of', 'from']."

In [10]:
def predict_masked_tokens_bench(text, tokenizer, model, top_k=3):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    mask_positions = torch.where(input_ids == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        output = model(input_ids)
    logits = output.logits

    predictions = []
    for mask_position in mask_positions:
        probs = logits[0, mask_position].softmax(dim=-1)
        top_k_values, top_k_indices = torch.topk(probs, top_k)
        top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices)

        mask_predictions = []
        for i in range(top_k):
            mask_predictions.append({
                "score": top_k_values[i].item(),
                "token": top_k_indices[i].item(),
                "token_str": top_k_tokens[i]
            })
        predictions.append(mask_predictions)

    return predictions


In [12]:
text = "1801—I have just returned from a [MASK] to my landlord—the solitary neighbour that I shall be troubled [MASK]." #original should be "visit" and "with"
predicted = predict_masked_tokens_bench(text, tokenizer, mmodel, top_k=5)
predicted

[[{'score': 0.7276092171669006, 'token': 3942, 'token_str': 'visit'},
  {'score': 0.1526014655828476, 'token': 3661, 'token_str': 'letter'},
  {'score': 0.015420506708323956, 'token': 2655, 'token_str': 'call'},
  {'score': 0.012914762832224369, 'token': 4471, 'token_str': 'message'},
  {'score': 0.011289449408650398, 'token': 2831, 'token_str': 'talk'}],
 [{'score': 0.44526180624961853, 'token': 2011, 'token_str': 'by'},
  {'score': 0.1372787207365036, 'token': 2055, 'token_str': 'about'},
  {'score': 0.13569167256355286, 'token': 2007, 'token_str': 'with'},
  {'score': 0.030658947303891182, 'token': 1997, 'token_str': 'of'},
  {'score': 0.02405373379588127, 'token': 2013, 'token_str': 'from'}]]

In [21]:
from itertools import product

combinations = list(product(*predicted))


In [25]:
combinations

[({'score': 0.7276092171669006, 'token': 3942, 'token_str': 'visit'},
  {'score': 0.44526180624961853, 'token': 2011, 'token_str': 'by'}),
 ({'score': 0.7276092171669006, 'token': 3942, 'token_str': 'visit'},
  {'score': 0.1372787207365036, 'token': 2055, 'token_str': 'about'}),
 ({'score': 0.7276092171669006, 'token': 3942, 'token_str': 'visit'},
  {'score': 0.13569167256355286, 'token': 2007, 'token_str': 'with'}),
 ({'score': 0.7276092171669006, 'token': 3942, 'token_str': 'visit'},
  {'score': 0.030658947303891182, 'token': 1997, 'token_str': 'of'}),
 ({'score': 0.7276092171669006, 'token': 3942, 'token_str': 'visit'},
  {'score': 0.02405373379588127, 'token': 2013, 'token_str': 'from'}),
 ({'score': 0.1526014655828476, 'token': 3661, 'token_str': 'letter'},
  {'score': 0.44526180624961853, 'token': 2011, 'token_str': 'by'}),
 ({'score': 0.1526014655828476, 'token': 3661, 'token_str': 'letter'},
  {'score': 0.1372787207365036, 'token': 2055, 'token_str': 'about'}),
 ({'score': 0.15

In [29]:
import re

replaced_texts = []
for combination in combinations:
    replaced_text = re.sub(r"\[MASK\]", "{}", text)
    replaced_text = replaced_text.format(*[pred["token_str"] for pred in combination])
    
    score = 1
    for pred in combination:
        score *= pred["score"]

    replaced_texts.append({
        "sequence": replaced_text,
        "score": score,
        # "tokens": combination
    })


In [30]:
replaced_texts

[{'sequence': '1801—I have just returned from a visit to my landlord—the solitary neighbour that I shall be troubled by.',
  'score': 0.3239765942796051},
 {'sequence': '1801—I have just returned from a visit to my landlord—the solitary neighbour that I shall be troubled about.',
  'score': 0.09988526252876095},
 {'sequence': '1801—I have just returned from a visit to my landlord—the solitary neighbour that I shall be troubled with.',
  'score': 0.0987305116500341},
 {'sequence': '1801—I have just returned from a visit to my landlord—the solitary neighbour that I shall be troubled of.',
  'score': 0.02230773264694552},
 {'sequence': '1801—I have just returned from a visit to my landlord—the solitary neighbour that I shall be troubled from.',
  'score': 0.017501718417162193},
 {'sequence': '1801—I have just returned from a letter to my landlord—the solitary neighbour that I shall be troubled by.',
  'score': 0.06794760420175772},
 {'sequence': '1801—I have just returned from a letter to